# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "Weather_data.csv"
city_df = pd.read_csv(city_data)
city_df

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind Speed,Clouds
0,0,kapaa,22.08,-159.32,82.31,74,13.87,75
1,1,bethel,41.37,-73.41,87.15,52,5.01,1
2,2,middlebury,44.02,-73.17,80.02,100,4.72,85
3,3,vaini,-21.20,-175.20,73.40,94,6.93,20
4,4,hobart,-42.88,147.33,37.71,84,6.93,52
...,...,...,...,...,...,...,...,...
550,550,jacareacanga,-6.22,-57.75,93.20,44,3.36,0
551,551,alvarado,18.77,-95.77,79.97,88,8.05,90
552,552,huarmey,-10.07,-78.15,66.97,66,11.90,14
553,553,itupiranga,-5.13,-49.33,87.80,66,6.93,40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lon"]]
humid = city_df["Humidity"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Ideal Conditions
# Temp between 73 and 85
# Cloudiness less than 10

temp_df = city_df.loc[city_df["Temp"] >= 73]
temp_df = temp_df.loc[temp_df["Temp"] <= 85]
ideal_df = temp_df.loc[temp_df["Clouds"] <= 10]
ideal_df

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind Speed,Clouds
19,19,dryden,49.78,-92.75,78.80,36,13.87,1
30,30,aksu,41.12,80.26,78.46,26,2.64,0
36,36,alugan,12.22,125.48,78.15,85,4.68,3
45,45,bengkulu,-3.80,102.27,76.14,82,6.64,2
56,56,shingu,33.73,135.98,74.01,88,2.91,0
61,61,bairiki,1.33,172.98,81.01,73,13.15,2
68,68,krestena,37.62,21.63,78.37,88,1.95,0
69,69,atuona,-9.80,-139.03,78.46,76,18.39,3
85,85,acarau,-2.89,-40.12,80.53,67,17.92,0
86,86,san policarpo,12.18,125.51,79.16,85,5.41,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df
len(hotel_df["City"])

53

In [7]:
hotel_name = []
#Creating a list of tuples for my  coordinates. in lesson plan day 3 activites 6 we create markers in a simple way with a list of tuples
coordinates = []
lat = hotel_df["Lat"].tolist()
lon = hotel_df["Lon"].tolist()
cities = hotel_df["City"].tolist()
c=1



for city in cities:
    print(c)
    target_coordinates = f"{lat[c]}, {lon[c]}"
    coorinate = (lat[c], lon[c])
    coordinates.append(coorinate)
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"
    c += 1
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel = response.json()
    try:
        hotel_name.append(hotel["results"][0]["name"])
    except:
        hotel_name.append("NaN")

    



    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>
#"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lon"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))